In [1]:
blob_relative_path = 'https_census.gov_programs-surveys/CENSUS_files/2023-06-01/apes' 
datetime = '2023-06-06'

In [2]:
# %pip install spark_excel

In [3]:
from notebookutils import mssparkutils  
import re
# import pyspark_excel
import pandas as pd
from pyspark.sql import SparkSession 
from pyspark import SparkContext
from pyspark.sql.types import StructType,StructField,StringType
from pyspark.sql.functions import split,col,substring,expr,first,isnull
# Azure storage access info 
blob_account_name = 'usafactsbronze' # replace with your blob name 
blob_container_name = 'bronze' # replace with your container name  
linked_service_name = 'bronze' # replace with your linked service name 

blob_sas_token = mssparkutils.credentials.getConnectionStringOrCreds(linked_service_name) 

# Allow SPARK to access from Blob remotely 
wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path) 
spark.conf.set('fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name), blob_sas_token) 


In [4]:
def get_file_paths(dir_path):
    file_paths= []
    files = mssparkutils.fs.ls(dir_path)
    for file in files:
        if file.isDir :
            file_paths.extend(get_file_paths(file.path))
        else:
            file_paths.append(file.path)

    return file_paths

In [5]:
all_file_paths = get_file_paths(wasbs_path)

In [6]:
len(all_file_paths)

In [7]:
# blob_name='download.bls.gov/pub/time.series/oe/oe.data.1.AllData'
# data=[]
# blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
# blob_data = blob_client.download_blob().readall().decode('utf-8')

In [8]:
# print(blob_data)

In [9]:
# text_data = blob_client.download_blob().readall().decode('utf-8')
# def csv_of_any_length(text_data):

#     data = []
#     lines = text_data.split('\n')
#     header_row = lines[0].split('\t')
#     header_length = len(header_row)
#     max_columns=header_length
#     maxlength = header_length

#     for idx, line in enumerate(lines):
#         row = line.split('\t')
#         row = [elem.strip() for elem in row]

#         maxlength = max(maxlength, len(row))
#         row += [''] * (max_columns - len(row)) # fill the row up to max_columns

#         # flag rows that are longer than the header row
#         if len(row) > header_length:
#             row.append('CORRUPT_FLAG')
#             maxlength = max(maxlength, len(row))

#         data.append(row)
#     max_columns=max(maxlength,max_columns)
#     for i in range(header_length, max_columns):
#         header_row.append(f'column{i}')
#         header_row.append('CORRUPT_FLAG')

#     df = pd.DataFrame(data[1:], columns=header_row)
#     # fliter df Down to needed

#     filtered_df = df.iloc[:, list(range(maxlength)) + [-1]]


#     return df

In [10]:
# # import pandas as pd
# # df =csv_of_any_length(text_data)
# # display(df)
# import pandas as pd
# lines = blob_data.replace('\r\n', '\r').replace('\n', '\r').split('\r') # <-- added to deal with inconsistent formatting of line breaks
# padding = [[] for _ in range(0, 5) ]
# # get a trimmed list of headers to parse and map
# headers=[val.strip() for val in lines[0].split('\t')]
# max_len=len(headers)
# for line in lines:
#     split_line=line.split('\t')
#     max_len=max(max_len,len(split_line))
#     data.append(line.split('\t')+padding)
# df=pd.DataFrame([dat[0:max_len] for dat in data],columns=headers)
# df.head()

In [11]:
# lines_rdd = spark.sparkContext.textFile(text_path)
# values_rdd = lines_rdd.map( lambda line: line.split('\t')).map(lambda words:[word.strip() for word in words])
# # column_names = values_rdd.first()
# text = values_rdd.map(tuple).collect()
# data_rdd = values_rdd.filter(lambda line: line.strip() != column_names)
# df = data_rdd.toDF(column_names)


In [12]:
def try_again(text_path,path):

    lines_rdd = spark.sparkContext.textFile(text_path)
    lines_rdd = lines_rdd.filter(lambda line: line.strip() != '')
    values_rdd = lines_rdd.map( lambda line: line.split('\t')).map(lambda words:[tuple(re.split(r'\s\s\s+',word.strip())) for word in words] )
    raw_data = values_rdd.collect()
    formatted_data =[]
    for text in raw_data:
        tuple_data = ()
        if len(text)>1 : 
            for word in text:
                tuple_data += word
            formatted_data.append(tuple_data)
        else:
            formatted_data.append(text[0])       
    columns =[col.strip() for col in formatted_data[0]]  
    text_data = []
    text_data.append(columns)
    text_data.extend(formatted_data[1:])
    if len(text_data[1]) != len(columns):
        
        if raw_data[2][-1]=='':
            text_data = []
            for text in raw_data :
                if text[-1]=='':
                    text_data.append(text[:-1])
                else:
                    text_data.append(text)
            columns =[re.sub(r'[^A-Za-z0-9]+','_',col.strip()) for col in text_data[0]]
            data = text_data[1:]
            for col in columns :
                if col.isnumeric():
                    cols = []
                    for ind in range(len(columns)):
                        cols.append('col_'+str(ind+1))
                    columns = cols
                    data = text_data
                    break
        else:
            columns = [] 

            for ind in range(len(text_data[1])):
                columns.append('col_'+str(ind+1))
            raw_data_col = text_data[0]
            for index in range(len(columns)-len(text_data[0])):
                raw_data_col.append(' ')
            data = []
            data.append(raw_data_col)
            data.extend(text_data[1:])
        df = spark.createDataFrame(data,columns)
    df.write.format('delta').mode('overwrite').option("overwriteSchema", "true").option("path",path).save()



In [13]:
def proceed_to_df(text_data):
    columns =[re.sub(r'[^A-Za-z0-9]+','_',col.strip()) for col in text_data[0] if col != '' ]
    data = text_data[1:]
    for col in columns :
        if col.isnumeric():
            cols = []
            for ind in range(len(columns)):
                cols.append('col_'+str(ind+1))
            columns = cols
            data =text_data
            break
    try:
        df = spark.createDataFrame(data,columns)
    except BaseException :
        new_data = []
        for text in text_data:
            data = text
            if len(text) != len(text_data[0]):
                temp_data =[]
                for word in text:
                    word = word.strip().split(' ')
                    temp_data.extend(word)
                data = tuple(temp_data)
            if len(data) == len(text_data[0]) :
                new_data.append(tuple(data))
        df = spark.createDataFrame(new_data,columns)
    return df

In [14]:
# text_data = blob_client.download_blob().readall().decode('utf-8').split('\n')
# lines_rdd = lines_rdd.split('\n')
import re
dfs = []
bad_record = []
bad_records = []
exclude = ['.exe','.txt','.contacts','.zip','.tar','.xml','.gz','README','.doc','.pdf','.gif','.sf','.xls']

for text_path in all_file_paths:

    if not any( excl in text_path.split('/')[-1] for excl in exclude) :
        if text_path.endswith('.csv'):
            df = spark.read.csv(text_path,header = True,inferschema = True)
            df_pandas = df.toPandas()
            dfs.append((df_pandas,text_path))
            continue
        # if text_path.endswith('.xls'):
        #     df = spark.read.format('com.crealytics.spark.excel').option('header','true').option('inferschema','true').load(text_path)
        #     dfs.append((df,text_path))
        #     continue
        try: 
            lines_rdd = spark.sparkContext.textFile(text_path)
            lines_rdd = lines_rdd.filter(lambda line: line.strip() != '')
            values_rdd = lines_rdd.map( lambda line: line.split('\t')).map(lambda words: [tuple(re.split(r'\s+',word.strip())) for word in words] ) 
            text_data = values_rdd.collect()
            formatted_data =[]
            for text in text_data:
                tuple_data = ()
                if len(text)>1 : 
                    for word in text:
                        tuple_data += word
                    formatted_data.append(tuple_data)
                else:
                    formatted_data.append(text[0])       
            column_names =[col for col in formatted_data[0]]  
            text_data = []
            text_data.append(column_names)
            text_data.extend(formatted_data[1:])
        except BaseException as e:
            try:
                values_rdd = lines_rdd.map( lambda line: line.split('\t')).map(lambda words: [word for word in words] )
                column_names = values_rdd.first()
                data_rdd = values_rdd.filter(lambda line: line != column_names)
                column_names =[re.sub(r'[^A-Za-z0-9]+','_',col.strip()) for col in column_names if col != '' ]
                df = data_rdd.toDF(column_names)
                dfs.append((df,text_path)) 
                continue
            except BaseException :
                bad_record.append(text_path)
                continue

        try:
            if len(text_data) == 1:
                schema = StructType([ StructField(re.sub(r'[^A-Za-z0-9]+','_',name.strip()),StringType(),True) for name in text_data[0] if name != ''])
                df= spark.createDataFrame([],schema)
                dfs.append((df,text_path))
                continue
            if len(column_names) <= 2 :
                if any(re.match(r'^[- ]*$',line[0]) for line in text_data[:10]) :
                    for ind,data in enumerate(text_data):
                        if re.match(r'^[- ]*$',data[0]) :
                            data =[]
                            columns = text_data[ind-1]
                            break
                
                    if len(columns) < len(text_data[ind+1]):
                        columns = [col.split() for col in columns][0]

                    if len(columns) == len(text_data[ind+1]):
                        data.append(columns)
                        data.extend(text_data[ind+1:]) 
                        df = proceed_to_df(data)
                        dfs.append((df,text_path))
                        continue

            if len(column_names) < len(text_data[1]):
                if text_data[1][-1] == '':
                    data =[]
                    data.append(column_names)
                    for text in text_data[1:] :
                        data.append(tuple(list(text)[:-1]))
                    text_data = data
                if len(column_names) == len(text_data[1]) :
                    df = proceed_to_df(text_data)
                    dfs.append((df,text_path))
                    continue

                if len(text_data[1]) == len(text_data[2]):
                    columns=[text.strip() for text in text_data[1] ]
                    df = spark.createDataFrame(text_data[2:],columns)
                    new_df = df.drop(col(''))
                    if len(column_names) == len(new_df.columns):
                        data=[]
                        columns=[text.strip() for text in text_data[1] if text !='']
                        data.append(column_names)
                        data.append(tuple(columns))
                        df_data =[tuple(row) for row in new_df.collect()]
                        data.extend(df_data)
                        df = proceed_to_df(data)
                        dfs.append((df,text_path))
                        continue    
                    else:
                        bad_record.append(text_path)
                        continue

            else:
                df = proceed_to_df(text_data)
                dfs.append((df,text_path)) 
                continue 

        except BaseException as e:
            bad_record.append(text_path)
            continue
    else :
        bad_records.append((text_path,text_path.split('/')[-1]))

In [15]:
connect_str = "DefaultEndpointsProtocol=https;AccountName=usafactsbronze;AccountKey=WEH1nIXRgYYjWEjRPC6szld67DOir5Jx46GenOM8bmA+yWQQLlzTeJv5fI02wVxtsW89pSU8lBFc+AStCz7fWw==;EndpointSuffix=core.windows.net"
from azure.storage.blob import BlobServiceClient
container_name='bronze'
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_client = blob_service_client.get_container_client(container_name)

In [16]:
from pyspark.sql import SparkSession 
from pyspark.sql.types import * 
from pyspark.sql.functions import split,col,substring,expr,first
# Azure storage access info 
blob_account_name = 'usafactssilver' # replace with your blob name 
blob_container_name = 'silver' # replace with your container name 
linked_service_name = 'silver' # replace with your linked service name 
# blob_relative_path = 'https_census.gov_programs-surveys/CENSUS_files/2023-06-01/apes'
blob_sas_token = mssparkutils.credentials.getConnectionStringOrCreds(linked_service_name) 

# Allow SPARK to access from Blob remotely 
wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path) 
spark.conf.set('fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name), blob_sas_token) 

In [17]:
# for file in files:
#     print(file.path)
#     df = spark.read.format("delta").load(file.path)
#     display(df)

In [18]:
for df in dfs :

    file_location = wasbs_path.split(wasbs_path.split('/')[-2])[0]+df[1].split('CENSUS_files/')[1]
    try:
        df[0].write.format('delta').mode('overwrite').option("overwriteSchema", "true").option("path",file_location).save()
    except BaseException as e:
        if df[1] not in bad_record:
            bad_record.append(df[1])

In [25]:
for record in bad_record:
    file_location = wasbs_path.split(wasbs_path.split('/')[-2])[0]+record.split('CENSUS_files/')[1]
    try:
        try_again(record,file_location)
    except BaseException as e :
        print(e)
        # bad_records.append((record,record.split('/')[-1]))

In [20]:
import pandas as pd

if len(bad_records)> 1:
    pandas_df = pd.DataFrame(bad_records,columns=["File_path","File_name"])
    file_name = blob_relative_path.split('/')[-1]
    bad_path = blob_relative_path.split(blob_relative_path.split('/')[-2])[0]+f'Bad_Records/{datetime}/{file_name}.bad_record.csv'
    blob_client = container_client.get_blob_client(f"{bad_path}")
    csv_file = pandas_df.to_csv(index=False)
    blob_client.upload_blob(csv_file,overwrite=True)

In [24]:
print(bad_records)